In [ ]:
from tqdm import tqdm
from collections import deque
import sys
import numpy as np
sys.path.append('../../../utils')
from TurboFJLT import *
from TurboFJLT_helpers import FJLT, TurboHDF5Reader

In [ ]:
datafile = "../../../data/fine_airfoil_cascade.h5"
reader = TurboHDF5Reader(datafile)
print(reader)

In [ ]:
num_snapshots = 500
snapshot_sequence = list(range(num_snapshots))

In [ ]:
q_mf = reader.load_meanflow()

### Approach used in notebooks

In [ ]:
def formQ(reader, seq_to_extract):
    num_dofs = reader.state_dim
    Q = np.empty((num_dofs, len(list(seq_to_extract))))
    reader.reset_chunked_loading(seq_to_extract, chunks_dim=50)
    for i, _ in enumerate(tqdm(seq_to_extract)):
        Q[:, i] = reader.load_next()-q_mf
    return Q

In [ ]:
%%time
Q = formQ(reader, snapshot_sequence)

### Check the I/O speed

In [ ]:
def formQ_v2(reader, seq_to_extract):
    num_dofs = reader.state_dim
    # Q = np.empty((num_dofs, len(list(seq_to_extract))), dtype=np.float64)
    Q = deque([])
    reader.reset_chunked_loading(seq_to_extract, chunks_dim=50)
    for i, _ in enumerate(tqdm(seq_to_extract)):
        col = reader.load_next()-q_mf
        Q.append(col)
    return Q

In [ ]:
%%time
Q = formQ_v2(reader, snapshot_sequence)

In [ ]:
%%time
Q_arr = np.empty((reader.state_dim, len(snapshot_sequence)))
for i in tqdm(range(100)):
    Q_arr[:, i] = Q.popleft()